## How to create a metric on top of a python function?

In [ ]:
import io
import pandas as pd
import requests
import zipfile

from datetime import datetime
from datetime import time
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn import ensemble
from sklearn.metrics import explained_variance_score, r2_score

from evidently import ColumnMapping

from evidently.base_metric import InputData
from evidently.metrics import RegressionQualityMetric
from evidently.metrics.custom_metric import CustomValueMetric
from evidently.report import Report
from evidently.renderers.html_widgets import WidgetSize
from evidently.test_suite import TestSuite
from evidently.tests.custom_test import CustomValueTest

### Load Data

In [ ]:
content = requests.get(
        "https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip",
        verify=False,
    ).content

with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(
        arc.open("hour.csv"),
        header=0,
        sep=",",
        parse_dates=["dteday"],
        index_col="dteday",
    )

raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row["hr"]))) + relativedelta(years=11),
    axis=1,
)
raw_data.sort_index(inplace=True)

reference = raw_data.loc["2023-01-01 00:00:00":"2023-01-28 23:00:00"]
current = raw_data.loc["2023-01-29 00:00:00":"2023-02-28 23:00:00"]

### Define column mapping

In [ ]:
target = "cnt"
prediction = "prediction"
numerical_features = ["temp", "atemp", "hum", "windspeed", "hr", "weekday"]
categorical_features = ["season", "holiday", "workingday"]

column_mapping = ColumnMapping()
column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

### Train regression model

In [ ]:
regressor = ensemble.RandomForestRegressor(random_state=0, n_estimators=50)
regressor.fit(reference[numerical_features + categorical_features], reference[target])

reference["prediction"] = regressor.predict(reference[numerical_features + categorical_features])
current["prediction"] = regressor.predict(current[numerical_features + categorical_features])

### Implement functions that return a single value

We use ```InputData``` as a function argument.

```InputData``` has the following structure:
```
    reference_data: Optional[pd.DataFrame]
    current_data: pd.DataFrame
    reference_additional_features: Optional[pd.DataFrame]
    current_additional_features: Optional[pd.DataFrame]
    column_mapping: ColumnMapping
    data_definition: DataDefinition
    additional_data: Dict[str, Any]
```

This basically means that you can access any attribute from ```InputData``` in your custom metric.

for more details checkout the code here: https://github.com/evidentlyai/evidently/blob/main/src/evidently/base_metric.py 

In [ ]:
def variance_func(data: InputData): 
    return explained_variance_score(data.current_data[data.column_mapping.target],
        data.current_data[data.column_mapping.prediction])

def r2_func(data: InputData): 
    return r2_score(data.current_data[data.column_mapping.target],
        data.current_data[data.column_mapping.prediction])

In the functions above we used ```current_data``` and ```column_mapping``` from ```InputData```.

For instance, if you need to acces ```reference_data```, ```additional_data``` or even ```data_definition``` you can access them all similarly.

### Wrap functions in a CustomValueMetric

In [ ]:
report = Report(
    metrics=[
        RegressionQualityMetric(),
        CustomValueMetric(func=r2_func, title="Current: R2 score", size=WidgetSize.HALF),
        CustomValueMetric(func=variance_func, title="Current: Variance", size=WidgetSize.HALF),
    ]
)

In [ ]:
report.run(
    reference_data=reference,
    current_data=current,
    column_mapping=column_mapping,
)

In [ ]:
report.show(mode='inline')

### Wrap a function in a CustomValueTest

In [ ]:
suite = TestSuite(tests=[CustomValueTest(func=r2_func, title='Current R2 score', lte=0.9, gte=0.7)])

In [ ]:
suite.run(
    reference_data=reference,
    current_data=current,
    column_mapping=column_mapping
)

In [ ]:
suite.show(mode='inline')

## How to use a Custom Metric in a monitoring dashboard?

Import relevant evidently modules to work with monitoring:

```{python}
from evidently.ui.workspace.cloud import CloudWorkspace

from evidently.ui.dashboards import CounterAgg
from evidently.ui.dashboards import DashboardPanelCounter
from evidently.ui.dashboards import PanelValue
from evidently.ui.dashboards import ReportFilter
from evidently.renderers.html_widgets import WidgetSize
```

Set up your workspace:
```{python}
workspace = CloudWorkspace(
        token="YOUR_TOKEN",
        url="https://app.evidently.cloud/"
        )
```

Create or load a project:
```{python}
project = ws.create_project("PROJECT_TITLE")
```

Create panels on top of custom metrics:
```{python}
project.dashboard.add_panel(
        DashboardPanelCounter(
            title="Custom Metric: R2",
            filter=ReportFilter(metadata_values={}, tag_values=[]),
            value=PanelValue(
                metric_id="CustomValueMetric",
                metric_args={"title": "R2 score"},
                field_path=CustomValueMetric.fields.value,
                legend="count",
            ),
            text="count",
            agg=CounterAgg.LAST,
            size=WidgetSize.HALF,
        )
)

project.dashboard.add_panel(
    DashboardPanelCounter(
        title="Custom Metric: Variance",
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        value=PanelValue(
            metric_id="CustomValueMetric",
            metric_args={"title": "Variance"},
            field_path=CustomValueMetric.fields.value,
            legend="count",
        ),
        text="count",
        agg=CounterAgg.LAST,
        size=WidgetSize.HALF,
    )
)
```